### 6. 상품 추가 기능

#### 6.1. WriteProduct.js (React)
파일 위치: c:/react/frontend/src/components/WriteProduct.js
```
import React, { useRef } from 'react';
import { useNavigate } from 'react-router';
import './main.css';


function WriteProduct() {
    const navigate = useNavigate();   // 화면주소이동
    const product_name = useRef();    // 폼 입력값 변경기능  (상품명과 연결)
    const price = useRef();
    const description = useRef();
    const img = useRef();
    return (
        <>
        <h2>상품 정보 등록</h2>
        <table>
            <tbody>
            <tr>
                <td>상품명</td>
                <td><input ref={product_name} /></td>   
            </tr>
            <tr>
                <td>가격</td>
                <td><input type='number' ref={price} /></td>
            </tr>
            <tr>
                <td>상품설명</td>
                <td><textarea rows='5' cols='60' ref={description} /></td>
            </tr>
            <tr>
                <td>상품이미지</td>
                <td>
                <input type='file' ref={img} />
                </td>
            </tr>
            <tr>
                <td colSpan='2' align='center'>
                <button type='button' onClick={() => {   // 버튼 클릭하면 아래 함수 실행해라
                    const form = new FormData();      // 폼 객체 만들기
                    form.append('product_name', product_name.current.value);
                                    //변수        데이터(폼에 입력한 값)
                    form.append('price', price.current.value);
                    form.append('description', description.current.value);
                    form.append('img', img.current.files[0]);
                    fetch('http://localhost/insert', {   // 패치: 서버호출
                        method: 'post',
                        encType: 'multipart/form-data',   // 첨부파일 포함 시 multipart form-data
                        body: form   // form내용을 보냄
                    }).then(() => {   // then: 콜백함수
                    navigate('/');    // 주소를 목록으로 가라
                    });
                }}>확인</button>
                <button onClick={() => navigate('/')}>목록</button>
                </td>
            </tr>
            </tbody>
        </table>
        </>
    );
};
export default WriteProduct;

```

#### 6.2 ListProduct.js(React)

```
...
import { useNavigate } from 'react-router';
...
function ListProduct(){
    const navigate = useNavigate();
    ...
}
function getList(url) {
    fetch(url)
      .then(response => { return response.json(); })
      .then(data => { setProductList(data); });
}
useEffect(() => {getList('http://localhost/list');}, []);

    ...
    <h2>상품목록</h2>
            <button onClick={() => navigate('/write')}>상품등록</button>
            <hr />
        
```


#### 6.3. App.js (React)
```
...
import WriteProduct from './components/WriteProduct';

function App(){ ...
  return(
        ...
        <Routes>
          ...
          <Route path='/wrtie' element={<WriteProduct />} />
          <Route path='*' element={<ListProduct />} />

```


#### 6.4. config/urls.py (Django)
```
urlpatterns = [
    ...
    path('insert', views.insert),
]
```

#### 6.5. shop/static , shop/static/images 디렉토리 추가

#### 6.6. shop/views.py (Django)
```
...
import os.path
from django.views.decorators.csrf import csrf_exempt

UPLOAD_DIR = os.path.dirname(__file__) + '/static/images/'

...
@csrf_exempt
def insert(request):
    if "img" in request.FILES:
        file = request.FILES["img"]
        file_name = file._name
        fp = open("%s%s" % (UPLOAD_DIR, file_name), "wb")
        for chunk in file.chunks():
            fp.write(chunk)
        fp.close()
    else:
        file_name = "-"
    row = Product(product_name=request.POST["product_name"], description=request.POST["description"],
                  price=request.POST["price"], filename=file_name)
    row.save()


```

### 7. 상품 검색 기능

#### 7.1. ListProduct.js (React)
파일 위치: c:/react/frontend/src/components/ListProduct.js
```
import React, { useEffect, useState, useRef } from 'react';

function ListProduct(){
    ...
    const product_name = useRef();
    
    ...
    
    useEffect(() => {getList('http://localhost/list');}, []);
    
    return(
    ...
        <h2>상품목록</h2>
            상품명: <input name='product_name' ref={product_name} />
            <button type='button' onClick={() => {
                getList(`http://localhost/list?product_name=${product_name.current.value}`)
                
            }}>조회</button>
            <br /><br />
    )}
```

#### 7.2. shop/views.py (Django)
```
...
def list(request):
    try:
        product_name = request.GET["product_name"]
    except:
        product_name = ""
    items = Product.objects.filter(product_name__contains=product_name).order_by("-product_name")
    ...
```

### 8. 상품 상세 기능

#### 8.1. ProductItem.js (React)
파일 위치: c:/react/frontend/src/components/ProductItem.js

```
...
import { Link } from 'react-router-dom';
...
            <div style={{ margin: '5px'}}>
                <span dangerouslySetInnerHTML={{__html:img}}></span>
                <Link to={`/detail/${product_code}`}>
                상품명: {product_name}<br />
                가격: {price}원
                </Link>
                ...
```

#### 8.2. DetailProduct.js (React)
파일 위치: c:/react/frontend/src/components/DetailProduct.js

```
import React, { useRef, useEffect, useState } from 'react';
import './main.css';
import { useNavigate } from 'react-router-dom';

function useFetch(url) {
    const [data, setData] = useState(null);
    const [loading, setLoading] = useState(true);
    useEffect(() => {
        fetch(url)
        .then(response => {
            return response.json();
        })
        .then(data => {
            setData(data);
            setLoading(false);
        })
    }, []);
    return [data, loading];
}

function DetailProduct() {
    const paths = window.location.href.split('/');
    const url = 'http://localhost/' + paths[paths.length - 2] + '/' + paths[paths.length - 1];
    const [data, loading] = useFetch(url);
    const navigate = useNavigate();
    const product_name = useRef();
    const price = useRef();
    const description = useRef();
    const img = useRef();
    if (loading) {
        return (
        <div>loading</div>
        )
    } else {
        let src='';
        let image_url = '';
        if (data.filename !== '-') {
        src = `http://localhost/static/images/${data.filename}`;
        image_url = `<img src=${src} width='300px' height='300px' />`;
        } else {
        image_url='';
        }
        return (
        <>
            <table>
            <tbody>
                <tr>
                <td>상품명</td>
                <td><input ref={product_name} defaultValue={data.product_name} /></td>
                </tr>
                <tr>
                <td>가격</td>
                <td><input type='number' ref={price} defaultValue={data.price} /></td>
                </tr>
                <tr>
                <td>상품설명</td>
                <td><textarea rows='5' cols='60' ref={description} defaultValue={data.description} /></td>
                </tr>
                <tr>
                <td>상품이미지</td>
                <td>
                    <span dangerouslySetInnerHTML={{ __html: image_url }}></span>
                    <br />
                    <input type='file' ref={img} />
                </td>
                </tr>
                <tr>
                <td colSpan='2' align='center'>
                    <button onClick={() => navigate('/')}>목록</button>
                </td>
                </tr>
            </tbody>
            </table>
        </>
        );
    }
}
export default DetailProduct;

```



#### 8.3. App.js (React)
파일 위치: c:/react/frontend/src/App.js
```
...
import DetailProduct from './components/DetailProduct';
...
function App(){
  console.warn = function no_console() {};  // 경고문제거
  return(
    <>
      <BrowserRouter>
        <Routes>
            ...
            <Route path='/detail/:product_code' element={<DetailProduct />} />
  )}

```

#### 8.4. config/urls.py (Django)
```
urlpatterns = [
    ...
    path('detail/<int:product_code>', views.detail),
]
```

#### 8.5. shop/views.py (Django)
```
def detail(request, product_code):
    row = Product.objects.get(product_code=product_code)
    serializer = ps.ProductSerializer(row)
    return HttpResponse(simplejson.dumps(serializer.data))
```

### 9. 상품 수정 기능

#### 9.1. DetailProduct.js (React)
파일 위치: c:/react/frontend/src/components/DetailProduct.js
```
    ...
                <td colSpan='2' align='center'>
                    <button type='button' onClick={() => {
                    const form = new FormData();
                    form.append('product_code', data.product_code);
                    form.append('product_name', product_name.current.value);
                    form.append('price', price.current.value);
                    form.append('description', description.current.value);
                    form.append('img', img.current.files[0]);
                    fetch('http://localhost/update', {
                        method: 'post',
                        encType: 'multipart/form-data',
                        body: form
                    }).then(() => {
                        navigate('/');
                    });
                    }
                    }>수정</button>
                    &nbsp;
                    <button onClick={() => navigate('/')}>목록</button>
                </td>
```

#### 9.2. config/urls.py (Django)
```
urlpatterns = [
    path('update', views.update),
]
```

#### 9.3. shop/views.py (Django)
```
...
@csrf_exempt
def update(request):
    product_code = request.POST['product_code']
    row_src = Product.objects.get(product_code=product_code)
    filename = row_src.filename
    if "img" in request.FILES:
        file = request.FILES["img"]
        filename = file._name
        fp = open("%s%s" % (UPLOAD_DIR, filename), "wb")
        for chunk in file.chunks():
            fp.write(chunk)
        fp.close()
    row_new = Product(product_code=product_code,
                      product_name=request.POST["product_name"],
                      description=request.POST["description"],
                      price=request.POST["price"],
                      filename=filename)
    row_new.save()
```